In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import time
pd.set_option("display.max_columns", None)

In [2]:
WD = "data"

In [3]:
raw_df = pd.read_csv(WD + "/archive/NIFTY50_all.csv", parse_dates=True)

In [4]:
raw_df["Symbol"].replace({"INFOSYSTCH": "INFY",
                          "KOTAKMAH": "KOTAKBANK",
                          "HINDLEVER": "HINDUNILVR",
                          "UTIBANK": "AXISBANK"}, inplace=True)

In [5]:
index_all_df = raw_df.pivot(index="Date", columns="Symbol", values="Close")

In [6]:
topk = pd.read_csv(WD + '/top10nifty50_230621.csv').head(9)
# k = 10

In [7]:
index_df = index_all_df[topk['SYMBOL']].copy()

In [8]:
index_df.fillna(method = 'bfill', inplace=True)

In [9]:
index_df = index_df[index_df.index.slice_indexer("2005-01-07")]

In [10]:
def AdjustStockSplits(data_df, cols = []):
    l = data_df.shape[0]
    result = pd.DataFrame()
    for col in data_df.columns:
#         print(col)
        result[col] = data_df[col].copy()
        if (len(cols) != 0 and col not in cols):
            continue
        factor = 1
        last_close = data_df[col][l - 1]
        for i in range(l):
            curr = l - i - 1
            if (last_close * factor < 0.6 * data_df[col][curr] ):
                factor = factor *  data_df[col][curr] / last_close
                print('split', col, data_df[col][curr], factor, last_close)
            result[col][curr] = (data_df[col][curr] / factor)
            result[col][curr] = data_df[col][curr] / factor
            last_close = result[col][curr]
#             print(data_df[col][curr], factor, last_close)
#         break
    return result
    

In [11]:
index_df = AdjustStockSplits(index_df)

split RELIANCE 1645.4 2.0112455690013444 818.1
split RELIANCE 2194.7 8.345758072013917 528.9011030752401
split HDFCBANK 2187.75 1.9869669860587622 1101.05
split HDFCBANK 2519.7 19.661766684358625 254.6343263627014
split INFY 1434.25 1.9456691311130707 737.15
split INFY 1975.05 7.548897276749774 509.05366393585496
split INFY 4349.85 116.56160069377809 281.71001962761125
split INFY 3385.65 27348.90761583446 14.429709183719028
split HDFC 3008.95 4.84377012234385 621.2
split ICICIBANK 1794.1 4.953340695748206 362.2
split TCS 3514.1 2.018379713391344 1741.05
split TCS 778.0 8.144572754105939 192.80316652912552
split TCS 1899.7 135.72602072222847 113.99615769064606
split KOTAKBANK 1425.6 2.016122189223589 707.1
split KOTAKBANK 864.6 7.729011898731478 225.53196548821552
split KOTAKBANK 464.75 136.93248427402247 26.232331203070895
split AXISBANK 1985.75 5.022764638927533 395.35
split LT 2446.4 2.0116766713263714 1216.1
split LT 2589.6 8.297141609897611 627.859346386527


In [12]:
# l = index_all_df.shape[0]
# for i in range(l):
#     print(i, index_all_df.index[i], index_all_df.LT.iloc[i])

In [13]:
# for col in index_all_df.columns:
#     print(index_all_df.loc[["2004-06-22", "2004-06-23"]][col])